<a href="https://colab.research.google.com/github/abyalex15/Medical/blob/main/MEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# prompt: upload csv file and know the path

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Get the path of the uploaded file
file_path = next(iter(uploaded))
print(f"The uploaded file is located at: {file_path}")

Saving mock_patient_data.csv to mock_patient_data.csv
User uploaded file "mock_patient_data.csv" with length 2151 bytes
The uploaded file is located at: mock_patient_data.csv


In [5]:
import pandas as pd

# Load data
df = pd.read_csv("mock_patient_data.csv")

# Respiration Rate Scoring
def score_respiration(rate):
    if rate <= 8:
        return 3
    elif 9 <= rate <= 11:
        return 1
    elif 12 <= rate <= 20:
        return 0
    elif 21 <= rate <= 24:
        return 2
    else:
        return 3

# SpO2 Scoring (Scale 1 assumed; Air/O2 can be handled separately)
def score_spo2(spo2, oxygen_type):
    if oxygen_type == 'Air':
        if spo2 <= 91:
            return 3
        elif 92 <= spo2 <= 93:
            return 2
        elif 94 <= spo2 <= 95:
            return 1
        else:
            return 0
    else:  # On Oxygen
        if spo2 <= 94:
            return 3
        elif 95 <= spo2 <= 96:
            return 2
        elif spo2 == 97:
            return 1
        else:
            return 0

# Heart Rate (Pulse) Scoring
def score_heart_rate(hr):
    if hr <= 40:
        return 3
    elif 41 <= hr <= 50:
        return 2
    elif 51 <= hr <= 100:
        return 0
    elif 101 <= hr <= 110:
        return 1
    elif 111 <= hr <= 130:
        return 2
    else:
        return 3

# Systolic Blood Pressure Scoring
def score_bp(bp):
    if bp <= 90:
        return 3
    elif 91 <= bp <= 100:
        return 2
    elif 101 <= bp <= 110:
        return 1
    elif 111 <= bp <= 219:
        return 0
    else:
        return 3

# Temperature Scoring (in Fahrenheit)
def score_temp(temp):
    if temp <= 95.0:
        return 3
    elif 95.1 <= temp <= 96.8:
        return 1
    elif 96.9 <= temp <= 100.4:
        return 0
    elif 100.5 <= temp <= 102.2:
        return 1
    else:
        return 2

# Consciousness Scoring
def score_consciousness(state):
    return 0 if state.strip().lower() == 'alert' else 3

# Apply scoring
df["Score_Respiration"] = df["Respiration Rate"].apply(score_respiration)
df["Score_SpO2"] = df.apply(lambda x: score_spo2(x["SpO2"], x["Oxygen Type"]), axis=1)
df["Score_HR"] = df["Heart Rate"].apply(score_heart_rate)
df["Score_BP"] = df["Systolic BP"].apply(score_bp)
df["Score_Temp"] = df["Temperature"].apply(score_temp)
df["Score_Consciousness"] = df["Consciousness"].apply(score_consciousness)

# Total MEWS Score
df["MEWS_Total"] = df[[
    "Score_Respiration", "Score_SpO2", "Score_HR",
    "Score_BP", "Score_Temp", "Score_Consciousness"
]].sum(axis=1)

# Output
print(df[["Time", "MEWS_Total"] + [col for col in df.columns if "Score_" in col]])

# Escalation logic (Optional)
def clinical_response(score):
    if score == 0:
        return "Routine monitoring"
    elif 1 <= score <= 4:
        return "Inform team leader"
    elif score >= 5 and score <= 6:
        return "Urgent: Notify doctor, consider ICU"
    elif score >= 7:
        return "Emergency: Call RRT"
    return "Invalid"

df["Response"] = df["MEWS_Total"].apply(clinical_response)

print("\n--- Clinical Responses ---")
print(df[["Time", "MEWS_Total", "Response"]])


     Time  MEWS_Total  Score_Respiration  Score_SpO2  Score_HR  Score_BP  \
0   00:00           4                  0           1         0         0   
1   00:01           3                  0           3         0         0   
2   00:02           5                  2           3         0         0   
3   00:03           3                  2           0         0         1   
4   00:04           5                  0           3         2         0   
5   00:05           5                  0           0         0         2   
6   00:06           4                  2           0         0         2   
7   00:07           0                  0           0         0         0   
8   00:08           3                  0           2         1         0   
9   00:09           5                  0           3         0         2   
10  00:10           5                  0           3         0         2   
11  00:11           3                  0           1         2         0   
12  00:12   